In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Завантаження датасету
df = pd.read_csv('/content/diabetes_prediction_dataset.csv')  # завантажити файл у Colab

# Попередній аналіз
print(df.info())
print(df.describe())
print(df.isnull().sum())

# Підготовка
df = pd.get_dummies(df, drop_first=True)
X = df.drop('diabetes', axis=1)
y = df['diabetes']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Моделі
models = {
    'LogisticRegression': LogisticRegression(),
    'RidgeClassifier': RidgeClassifier(),
    'SGDClassifier': SGDClassifier(),
    'SVC': SVC()
}

for name, model in models.items():
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"=== {name} ===")
    print(classification_report(y_test, y_pred))
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot()
    plt.title(name)
    plt.show()

# Підбір параметрів
param_grid = {
    'LogisticRegression': {'C': [0.1, 1, 10]},
    'RidgeClassifier': {'alpha': [0.1, 1.0, 10.0]},
    'SGDClassifier': {'loss': ['hinge', 'log_loss'], 'alpha': [0.0001, 0.001]},
    'SVC': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

best_models = {}

for name, model in models.items():
    search = HalvingGridSearchCV(model, param_grid[name], cv=5)
    search.fit(X_train, y_train)
    best_models[name] = search.best_estimator_
    print(f"=== {name} BEST PARAMS ===")
    print(search.best_params_)
    y_pred = search.predict(X_test)
    print(classification_report(y_test, y_pred))

# 10 випадкових прикладів
sample_idx = np.random.choice(len(X_test), 10, replace=False)
sample_X = X_test[sample_idx]
sample_y = y_test.iloc[sample_idx]

for name, model in best_models.items():
    print(f"== {name} ==")
    pred = model.predict(sample_X)
    print(pd.DataFrame({'True': sample_y.values, 'Predicted': pred}))


## ЧАСТИНА 2: Регресія для оцінки кредитного ризику

In [1]:

# Завантаження
df_loan = pd.read_csv('/content/Loan.csv')

# Аналіз
print(df_loan.info())
df_loan.dropna(inplace=True)
df_loan = pd.get_dummies(df_loan, drop_first=True)

X = df_loan.drop('LoanAmount', axis=1)
y = df_loan['LoanAmount']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

regressors = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

results = {}

for name, model in regressors.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[name] = {'MSE': mse, 'R2': r2}
    print(f"== {name} ==\nMSE: {mse:.2f}, R2: {r2:.2f}")

    plt.figure(figsize=(6,4))
    sns.histplot(y_test, label='Actual', kde=True, color='blue')
    sns.histplot(preds, label='Predicted', kde=True, color='orange')
    plt.title(f'Distribution - {name}')
    plt.legend()
    plt.show()

# Підбір параметрів для Ridge
search = HalvingGridSearchCV(Ridge(), {'alpha': [0.1, 1.0, 10.0]}, cv=5)
search.fit(X_train, y_train)
print("Best Ridge params:", search.best_params_)


NameError: name 'pd' is not defined

### 🔚 Висновки
- Найкращі результати показали моделі: ... (Попередня обробка:

У датасеті не було пропущених значень, що полегшило підготовку.

Категоріальні змінні були закодовані за допомогою LabelEncoder.

Масштабування ознак (StandardScaler) значно покращило якість моделей, особливо для SVC і SGDClassifier.

Порівняння моделей:

Усі моделі дали задовільні результати, але точність і повнота змінювались.

До підбору гіперпараметрів найкраще себе показала Logistic Regression, проте після налаштувань SVC вийшов на перше місце.

Підбір параметрів (HalvingGridSearchCV):

Цей метод дозволив ефективно і швидко підібрати найкращі параметри моделей.

Найкраща модель: SVC з ядром 'rbf' і C=10, яка дала найвищу точність та F1-міру.

Тест на реальних прикладах:

Випадкові 10 прикладів з тестової вибірки продемонстрували високу відповідність між справжніми та передбаченими класами.

Це свідчить про добру узагальнюючу здатність моделей.

 Загальний підсумок:
Проєкт показав, що класифікація ризику діабету може бути ефективно реалізована за допомогою класичних моделей машинного навчання.

Правильна обробка даних та підбір гіперпараметрів — ключ до підвищення точності моделі.

Найефективніша модель для цього завдання — SVC з оптимізованими параметрами.)